## Imports

In [ ]:
from pymongo import MongoClient
from bson.json_util import loads, dumps
import json
import nltk
from nltk.corpus import stopwords
import collections

## Stopwords

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

## Connecting to database

In [ ]:
client = MongoClient('mongodb://localhost:27017')
db = client['twitter']
collection = db['tweets']

## Get data

In [ ]:
#data = json.loads(dumps(collection.find()))


with open("5000.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())

## Rausfiltern der Tweet-Inhalte
{
    ('word', 'year'): #count
}

In [ ]:
text_strings = []
for d in data:
    if "text" in d.keys():
        text_strings += [d["text"]]

words_dict = {}
for d in data:
    if not "text" in d.keys() or not "user" in d.keys() or not "created_at" in d["user"].keys() or d["user"]["created_at"] == None:
        continue
    text = d["text"].lower()
    created = d["user"]["created_at"][-4:]
    
    for word in text.split(' '):
        pair = (word, created)
        if pair in words_dict.keys():
            words_dict[pair] += 1
        else:
            words_dict[pair] = 1
words_dict

## Wort-Jahr-Anzahl

In [ ]:
ignore_words = stop_words + ['rt', '', '-', '&amp;', 'like', 'get', 'one', 'via', 'new', 'i\'m', 'rt']
words = [(k,v) for k, v in sorted(words_dict.items(), key=lambda item: -item[1]) if k[0] not in ignore_words]
words[:10]

## Top-10 Wörter pro Jahr der Account-Erstellung

In [ ]:
result = {}

for pair in words_dict.keys():
    word, year = pair
    amount = (word, words_dict[pair])
    
    if year in result.keys():              
        result[year] = result[year] + [(word, words_dict[pair])]
            
    else:
        result[year] = [(word, words_dict[pair])]

for year in sorted(result.keys()):
    print(year)
    value = result[year]
    r = [(k,v) for k, v in sorted(value, key=lambda item: -item[1]) if k not in ignore_words]
    for x in r[:10]:
        print("    ", x[0],":", x[1])

## Durchschnittliches Alter der Accounts

In [ ]:
years = [int(year) for d in data if "user" in d.keys() for year in [d["user"]["created_at"][-4:]]]
sum(years) / len(years)